# Laboratorio 04 - COVID

## Carga de datos csv en spark desde un bucket S3.

In [4]:
from pyspark.sql import SparkSession

spark=SparkSession.builder.appName('covid_lab').getOrCreate()
data=spark.read.csv('s3://acanomdatasets/covid/Casos_positivos_de_COVID-19_en_Colombia.csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Codigo DIVIPOLA: integer (nullable = true)
 |-- Ciudad de ubicación: string (nullable = true)
 |-- Departamento o Distrito : string (nullable = true)
 |-- atención: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País de procedencia: string (nullable = true)
 |-- FIS: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha diagnostico: timestamp (nullable = true)
 |-- Fecha recuperado: string (nullable = true)
 |-- fecha reporte web: timestamp (nullable = true)

## borrar y crear algunas columnas
- Columnas borradas: Codigo DIVIPOLA, fecha reporte web
- Columnas creadas: Días entre notificación y diagnostico, y Menor de 18

### Borrrar columnas

In [6]:
data2=data.drop('Codigo DIVIPOLA', 'fecha reporte web')
data2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Ciudad de ubicación: string (nullable = true)
 |-- Departamento o Distrito : string (nullable = true)
 |-- atención: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País de procedencia: string (nullable = true)
 |-- FIS: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha diagnostico: timestamp (nullable = true)
 |-- Fecha recuperado: string (nullable = true)

### Crear columnas

In [7]:
# días entre fecha de notificación (SIVIGILA) y fecha de diagnostico.
data2 = data2.withColumn(
    "Días entre notificación y diagnostico", 
    (data2["Fecha diagnostico"].cast("long") - data2["Fecha de notificación"].cast("long"))/86400
)
data2.select("Fecha de notificación", "Fecha diagnostico", "Días entre notificación y diagnostico").show(10)
data2.select("Fecha de notificación", "Fecha diagnostico", "Días entre notificación y diagnostico").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------------------+-------------------+-------------------------------------+
|Fecha de notificación|  Fecha diagnostico|Días entre notificación y diagnostico|
+---------------------+-------------------+-------------------------------------+
|  2020-03-02 00:00:00|2020-03-06 00:00:00|                                  4.0|
|  2020-03-06 00:00:00|2020-03-09 00:00:00|                                  3.0|
|  2020-03-07 00:00:00|2020-03-09 00:00:00|                                  2.0|
|  2020-03-09 00:00:00|2020-03-11 00:00:00|                                  2.0|
|  2020-03-09 00:00:00|2020-03-11 00:00:00|                                  2.0|
|  2020-03-10 00:00:00|2020-03-11 00:00:00|                                  1.0|
|  2020-03-08 00:00:00|2020-03-11 00:00:00|                                  3.0|
|  2020-03-09 00:00:00|2020-03-11 00:00:00|                                  2.0|
|  2020-03-08 00:00:00|2020-03-11 00:00:00|                                  3.0|
|  2020-03-12 00

In [8]:
# Menores de 18 años
data2 = data2.withColumn("Menor de 18", data2["Edad"] < 18)
data2.select("Edad", "Menor de 18").filter(data2["Menor de 18"]==True).show(10)
data2.select("Edad", "Menor de 18").filter(data2["Menor de 18"]==True).count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----------+
|Edad|Menor de 18|
+----+-----------+
|   9|       true|
|  16|       true|
|  17|       true|
|  15|       true|
|   9|       true|
|  15|       true|
|   3|       true|
|   9|       true|
|  16|       true|
|  16|       true|
+----+-----------+
only showing top 10 rows

553

In [9]:
data2.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- ID de caso: integer (nullable = true)
 |-- Fecha de notificación: timestamp (nullable = true)
 |-- Ciudad de ubicación: string (nullable = true)
 |-- Departamento o Distrito : string (nullable = true)
 |-- atención: string (nullable = true)
 |-- Edad: integer (nullable = true)
 |-- Sexo: string (nullable = true)
 |-- Tipo: string (nullable = true)
 |-- Estado: string (nullable = true)
 |-- País de procedencia: string (nullable = true)
 |-- FIS: string (nullable = true)
 |-- Fecha de muerte: string (nullable = true)
 |-- Fecha diagnostico: timestamp (nullable = true)
 |-- Fecha recuperado: string (nullable = true)
 |-- Días entre notificación y diagnostico: double (nullable = true)
 |-- Menor de 18: boolean (nullable = true)

## Realizar filtrados de datos por alguna información que le parezca interesante
- Filtro 1: Personas en Antioquia con el virus importado
- Filtro 2: Menores de 18 años fallecidos
- Filtro 3: Personas en Antioquia recuperados

In [11]:
filtro1 = data2.filter(
   (data2["Departamento o Distrito "] == "Antioquia") & (data2["Tipo"] == "Importado")
).select(
    "Ciudad de ubicación", "Edad", "Sexo",  "Tipo", "Estado"
).orderBy("Edad")
filtro1.show()
filtro1.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+----+---------+------+
|Ciudad de ubicación|Edad|Sexo|     Tipo|Estado|
+-------------------+----+----+---------+------+
|             Itagüí|   9|   F|Importado|  Leve|
|           Medellín|  15|   F|Importado|  Leve|
|           Envigado|  20|   M|Importado|  Leve|
|           Sabaneta|  21|   M|Importado|  Leve|
|           Medellín|  21|   F|Importado|  Leve|
|           Medellín|  21|   F|Importado|  Leve|
|           Medellín|  21|   M|Importado|  Leve|
|           Envigado|  21|   F|Importado|  Leve|
|           Medellín|  22|   F|Importado|  Leve|
|             Retiro|  23|   F|Importado|  Leve|
|           Medellín|  23|   F|Importado|  Leve|
|           Medellín|  24|   F|Importado|  Leve|
|           Medellín|  24|   F|Importado|  Leve|
|             Guarne|  25|   F|Importado|  Leve|
|           Medellín|  25|   M|Importado|  Leve|
|            Cocorná|  25|   M|Importado|  Leve|
|           Envigado|  26|   M|Importado|  Leve|
|           Medellín

In [12]:
filtro2 = data2.filter(
   ((data2["Menor de 18"] == True) & (data2["Estado"]=="Fallecido"))
).select(
    "Ciudad de ubicación", "Edad", "Menor de 18", "Sexo",  "Tipo", "Estado"
).orderBy("Edad")
filtro2.show()
filtro2.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+-----------+----+-----------+---------+
|Ciudad de ubicación|Edad|Menor de 18|Sexo|       Tipo|   Estado|
+-------------------+----+-----------+----+-----------+---------+
|             Samacá|   1|       true|   F| En estudio|Fallecido|
|       Cundinamarca|   1|       true|   M| En estudio|Fallecido|
|            Popayán|   3|       true|   F|Relacionado|Fallecido|
+-------------------+----+-----------+----+-----------+---------+

3

In [13]:
filtro3 = data2.filter(
   ((data2["Departamento o Distrito "] == "Antioquia") & (data2["atención"]=="Recuperado"))
).select(
    "Ciudad de ubicación", "Edad", "Menor de 18", "Sexo", "atención"
).orderBy("Edad")
filtro3.show()
filtro3.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----+-----------+----+----------+
|Ciudad de ubicación|Edad|Menor de 18|Sexo|  atención|
+-------------------+----+-----------+----+----------+
|           Medellín|   1|       true|   M|Recuperado|
|              Bello|   4|       true|   M|Recuperado|
|           Medellín|   5|       true|   F|Recuperado|
|           Medellín|   5|       true|   M|Recuperado|
|             Sonsón|   5|       true|   M|Recuperado|
|           Medellín|   8|       true|   F|Recuperado|
|             Itagüí|   9|       true|   F|Recuperado|
|           Medellín|  12|       true|   M|Recuperado|
|           Frontino|  15|       true|   M|Recuperado|
|           Medellín|  15|       true|   F|Recuperado|
|           Envigado|  18|      false|   F|Recuperado|
|           Medellín|  18|      false|   F|Recuperado|
|           Medellín|  19|      false|   F|Recuperado|
|           Medellín|  20|      false|   F|Recuperado|
|           Envigado|  20|      false|   M|Recuperado|
|         

## Realizar alguna agrupación y consulta de datos categorica, por ejemplo número de casos por región o por sexo/genero.
- Grupo 1: Personas agrupadas por país de procedencia Fallecidos
- Grupo 2: Personas agrupadas por Ciudad de ubicación en el departamento de Antioquia con su tipo de atención

In [14]:
grupo1 = data2.filter(data2["atención"] == "Fallecido").select(
    "atención", "estado", "País de procedencia"
).groupBy('País de procedencia', "atención").count().orderBy('count', ascending=False)
grupo1.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+-----+
| País de procedencia| atención|count|
+--------------------+---------+-----+
|            Colombia|Fallecido|  309|
|Estados Unidos de...|Fallecido|    4|
|              España|Fallecido|    4|
|             Ecuador|Fallecido|    1|
|              Panamá|Fallecido|    1|
|              Egipto|Fallecido|    1|
|           Venezuela|Fallecido|    1|
|            Alemania|Fallecido|    1|
|              Brasil|Fallecido|    1|
|             Turquía|Fallecido|    1|
+--------------------+---------+-----+

In [15]:
grupo2 = data2.filter(data2["Departamento o Distrito "] == "Antioquia").select(
    "atención", 'Ciudad de ubicación', 'Departamento o Distrito '
).groupBy('Ciudad de ubicación', "atención").count().orderBy('count', ascending=False)
grupo2.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+------------+-----+
|Ciudad de ubicación|    atención|count|
+-------------------+------------+-----+
|           Medellín|        Casa|  183|
|           Medellín|  Recuperado|  111|
|              Bello|        Casa|   30|
|              Bello|  Recuperado|   19|
|           Envigado|  Recuperado|   17|
|           Envigado|        Casa|   11|
|           Medellín|Hospital UCI|   10|
|           Rionegro|  Recuperado|    9|
|           Sabaneta|  Recuperado|    7|
|             Itagüí|  Recuperado|    7|
|             Itagüí|        Casa|    7|
|         Copacabana|        Casa|    7|
|        La Estrella|        Casa|    5|
|        La Estrella|  Recuperado|    4|
|           Medellín|   Fallecido|    3|
|           Medellín|    Hospital|    3|
|             Caldas|  Recuperado|    3|
|         Montebello|        Casa|    3|
|           Apartadó|        Casa|    2|
|           Apartadó|  Recuperado|    2|
+-------------------+------------+-----+
only showing top

## Grave los resultados en un bucket público en S3
Bucket: s3://acanomoutputs/covid

In [21]:
url = "s3://acanomoutputs/covid"

data2.coalesce(1).write.format("csv").option("header","true").save(url + "/data2")
filtro1.coalesce(1).write.format("csv").option("header","true").save(url + "/filtro1")
filtro2.coalesce(1).write.format("csv").option("header","true").save(url + "/filter2")
filtro3.coalesce(1).write.format("csv").option("header","true").save(url + "/filter3")
grupo1.coalesce(1).write.format("csv").option("header","true").save(url + "/grupo1")
grupo2.coalesce(1).write.format("csv").option("header","true").save(url + "/grupo2")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

# Conclusiones

Se observa que hay un gran número de personas en Antioquia (123) que importaron el virus de otros paises. 192 personas se han recuperado en Antioquia. Han muerto 3 menores de edad en Colombia. Por otra parte, basados en los paises de donde se ha importado el virus a Colombia, Estados Unidos y España presentan el mayor número de muertes. Medellín y Bello lideran el mayor número de casos en Antioquia.